In [43]:
import pandas as pd 


In [44]:
df = pd.read_csv('sub-data.csv')
df.head()

,POLVIEWS,PARTYID,YEAR
0,4,2,1974
1,5,0,1974
2,6,0,1974
3,6,0,1974
4,6,1,1974


In [45]:
#Renombrara las columnas de Polviews a orientacion politica y Partyid a partido
df.rename(columns={'POLVIEWS': 'orientacion politica', 'PARTYID': 'partido'}, inplace=True)
df.head()

,orientacion politica,partido,YEAR
0,4,2,1974
1,5,0,1974
2,6,0,1974
3,6,0,1974
4,6,1,1974


In [46]:
# Quearme con orientacion 
df=df.drop(df[df["orientacion politica"]==8].index)
df=df.drop(df[df["orientacion politica"]==9].index)
df=df.drop(df[df["orientacion politica"]==0].index)
#quedarme solo con 0 y 6 en PartyID
df = df[df["partido"].isin([0, 6])]
df

,orientacion politica,partido,YEAR
1,5,0,1974
2,6,0,1974
3,6,0,1974
7,5,0,1974
9,6,0,1974
...,...,...,...
3801,1,0,2018
3803,3,0,2018
3812,5,6,2018
3821,6,6,2018


In [47]:
df_1974 = df[df['YEAR'] == 1974]
df_2018 = df[df['YEAR'] == 2018]
df_1974.head()

,orientacion politica,partido,YEAR
1,5,0,1974
2,6,0,1974
3,6,0,1974
7,5,0,1974
9,6,0,1974


In [ ]:
# Función para el intervalo de confianza de una diferencia de medias, suponiendo distr. pivotal t de student
from scipy.stats import t    # Necesario para la definición de la función
import numpy as np
def interv_conf_dosmedias(y1, y2, m_var = True, alfa = 0.05):
    n1 = len(y1); n2 = len(y2)
    v1 = np.var(y1, ddof = 1); v2 = np.var(y2, ddof = 1)
    if m_var:
        df = n1+n2-2
        var_resta = (((n1-1)*v1+(n2-1)*v2)/(n1+n2-2))*(1/n1+1/n2)
    else:
        df = (v1/n1+v2/n2)**2/(v1**2/(n1**2*(n1-1))+v2**2/(n2**2*(n2-1)))
        var_resta = v1/n1+v2/n2
    se = np.sqrt(var_resta)
    t_a = t.isf(alfa/2,df)
    resta_medias = np.mean(y1)-np.mean(y2)
    interv_conf = resta_medias+np.array([-1, 1])*t_a*se
    confianza = 1-alfa
    return resta_medias, interv_conf, confianza, df

# Función para t-test de dos muestras ind.
from scipy.stats import ttest_ind
# Para echarle un oclayo a la documentación
help(ttest_ind)

In [49]:
grupo_0 = df_1974[df_1974["partido"] == 0]["orientacion politica"]
grupo_6 = df_1974[df_1974["partido"] == 6]["orientacion politica"]
resta_medias, intervalo, conf, grados_libertad = interv_conf_dosmedias(grupo_0, grupo_6, m_var=True, alfa=0.05)

print(f"Año 1974")
print(f"Diferencia de medias: {resta_medias}")
print(f"Intervalo de confianza al {conf*100:.1f}%: {intervalo}")
print(f"Grados de libertad: {grados_libertad:.2f}")

Año 1974
Diferencia de medias: -0.940645270699962
Intervalo de confianza al 95.0%: [-1.2636247  -0.61766584]
Grados de libertad: 330.00


In [50]:
grupo_0 = df_2018[df_2018["partido"] == 0]["orientacion politica"]
grupo_6 = df_2018[df_2018["partido"] == 6]["orientacion politica"]
resta_medias, intervalo, conf, grados_libertad = interv_conf_dosmedias(grupo_0, grupo_6, m_var=True, alfa=0.05)

print(f"Año 2018")
print(f"Diferencia de medias: {resta_medias}")
print(f"Intervalo de confianza al {conf*100:.1f}%: {intervalo}")
print(f"Grados de libertad: {grados_libertad:.2f}")

Año 2018
Diferencia de medias: -2.741934092477571
Intervalo de confianza al 95.0%: [-2.95530577 -2.52856242]
Grados de libertad: 618.00


Podemos ver que en 2018 como tenemos mas datos eso puede ayudar a que sea mas preciso , pero como vemos que la media entre ambos estan en un intervalo ma chico pero son valores altos , lo que nos dice que las medias son mas altos son mas altas entre amos grupos para el 2018 por lo que hoy en dia exiten los grupos mas polarizados.  Ahroa bien en 1974 como no tenemos la misma cantidad de datos (esto se puede deber a que antes los datos se registraban a hoja y papel por lo que se perdia mucha info) entonces su rango es mas amplio y no es tan preciso pero sus media no tiene tanta diferencia lo que nos dice que no habia mucha diferencia o polarizacion entre grupos

# Prueba de hipotesis 

1974

In [51]:
grupo_0 = df_1974[df_1974["partido"] == 0]["orientacion politica"]
grupo_6 = df_1974[df_1974["partido"] == 6]["orientacion politica"]
t_stat, p_val = ttest_ind(grupo_0, grupo_6, equal_var=True)  

print("Estadístico t:", t_stat)
print("Valor p:", p_val)


Estadístico t: -5.729211084761955
Valor p: 2.2741011426597366e-08


In [53]:
if p_val < 0.05:
    print("Rechazamos H₀: hay evidencia de diferencia en las medias.")
else:
    print("No se rechaza H₀: no hay evidencia suficiente para afirmar diferencia en las medias.")

Rechazamos H₀: hay evidencia de diferencia en las medias.


2018

In [52]:
grupo_0 = df_2018[df_2018["partido"] == 0]["orientacion politica"]
grupo_6 = df_2018[df_2018["partido"] == 6]["orientacion politica"]
t_stat, p_val = ttest_ind(grupo_0, grupo_6, equal_var=True)  

print("Estadístico t:", t_stat)
print("Valor p:", p_val)


Estadístico t: -25.235953181450288
Valor p: 4.009417938864369e-97


In [54]:
if p_val < 0.05:
    print("Rechazamos H₀: hay evidencia de diferencia en las medias.")
else:
    print("No se rechaza H₀: no hay evidencia suficiente para afirmar diferencia en las medias.")

Rechazamos H₀: hay evidencia de diferencia en las medias.


Como se rechaza en los dos casos las medias no son las mismas existe evidencia de la estadísticamente significativa de que la media de orientación política difiere entre los votantes de los partidos 0 y 6, tanto en 1974 como en 2018.

En cuanto al contexto histroico es importante resaltar que en1974, la diferencia ideológica entre estos dos grupos ya era significativa.

En 2018, esa diferencia sigue siendo significativa , pero puede ser incluso aun mas pues esta en un valor de e-97.



Me da mas informacion una prueba de hipotesis , por que puedes comparar y queremos tomar decisiones, pero si queremos esrtimar es mejor un intevalo de confianza.  Queremos una respuesta concreta